In [54]:
import random
import json
import pickle
from statistics import mode
import numpy as np
import nltk
import tensorflow as tf

from tensorflow.keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [55]:
lemmatizer = WordNetLemmatizer()

In [56]:
intents = json.loads(open('intents.json').read())

In [57]:
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]

In [58]:
for intent in intents["intents"]:
    print("Intent: {}".format(intent))
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        print("Word List: {}".format(word_list))
        words.extend(word_list)

        documents.append(((word_list), intent["tag"]))
        # print(documents)
        if(intent["tag"] not in classes):
            classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word.lower())
         for word in words if word not in ignore_letters]
words = sorted(list(set(words)))

Intent: {'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Hello there', 'Good morning', 'Good evening', 'how is it going', 'greetings', 'Tell me something'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?', 'Hey!']}
Word List: ['Hi']
Word List: ['How', 'are', 'you']
Word List: ['Is', 'anyone', 'there', '?']
Word List: ['Hello']
Word List: ['Good', 'day']
Word List: ['Hello', 'there']
Word List: ['Good', 'morning']
Word List: ['Good', 'evening']
Word List: ['how', 'is', 'it', 'going']
Word List: ['greetings']
Word List: ['Tell', 'me', 'something']
Intent: {'tag': 'name', 'patterns': ['What is your name?', 'What should I call you?', 'What do you go by?', "What's your name?", 'What should I call you?', 'Can you introduce yourself?'], 'responses': ['My name is Chatbot.', 'You can call me Chatbot.', "I'm Chatbot!", 'I go by the alias: Chatbot.']}
Word List: ['What', 'is', 'your', 'name', '?']
Word List: ['What', 'sh

In [59]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [60]:
training = []
output_empty = [0] * len(classes)
for document in documents:
    print("Document: {}".format(document))
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

Document: (['Hi'], 'greeting')
Document: (['How', 'are', 'you'], 'greeting')
Document: (['Is', 'anyone', 'there', '?'], 'greeting')
Document: (['Hello'], 'greeting')
Document: (['Good', 'day'], 'greeting')
Document: (['Hello', 'there'], 'greeting')
Document: (['Good', 'morning'], 'greeting')
Document: (['Good', 'evening'], 'greeting')
Document: (['how', 'is', 'it', 'going'], 'greeting')
Document: (['greetings'], 'greeting')
Document: (['Tell', 'me', 'something'], 'greeting')
Document: (['What', 'is', 'your', 'name', '?'], 'name')
Document: (['What', 'should', 'I', 'call', 'you', '?'], 'name')
Document: (['What', 'do', 'you', 'go', 'by', '?'], 'name')
Document: (['What', "'s", 'your', 'name', '?'], 'name')
Document: (['What', 'should', 'I', 'call', 'you', '?'], 'name')
Document: (['Can', 'you', 'introduce', 'yourself', '?'], 'name')
Document: (['Bye'], 'goodbye')
Document: (['See', 'you', 'later'], 'goodbye')
Document: (['Goodbye'], 'goodbye')
Document: (['I', 'am', 'leaving'], 'goodbye

In [61]:
random.shuffle(training)
training = np.array(training, dtype=object)

training_x = list(training[:, 0])
training_y = list(training[:, 1])

In [62]:
model = Sequential()
model.add(Dense(128, input_shape=(len(training_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(training_y[0]), activation="softmax"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               10880     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 8)                 520       
                                                                 
Total params: 19,656
Trainable params: 19,656
Non-trainable params: 0
_________________________________________________________________


In [63]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy",
              optimizer=sgd, metrics=["accuracy"])
history = model.fit(np.array(training_x), np.array(
    training_y), epochs=250, batch_size=5, verbose=1)

Epoch 1/250
13/13 [==============================] - 0s 2ms/step - loss: 2.1760 - accuracy: 0.0952
Epoch 2/250
13/13 [==============================] - 0s 2ms/step - loss: 2.0591 - accuracy: 0.2063
Epoch 3/250
13/13 [==============================] - 0s 2ms/step - loss: 1.8941 - accuracy: 0.4286
Epoch 4/250
13/13 [==============================] - 0s 2ms/step - loss: 1.7861 - accuracy: 0.3810
Epoch 5/250
13/13 [==============================] - 0s 2ms/step - loss: 1.6631 - accuracy: 0.4444
Epoch 6/250
13/13 [==============================] - 0s 2ms/step - loss: 1.5161 - accuracy: 0.5714
Epoch 7/250
13/13 [==============================] - 0s 2ms/step - loss: 1.4388 - accuracy: 0.5714
Epoch 8/250
13/13 [==============================] - 0s 2ms/step - loss: 1.2350 - accuracy: 0.5238
Epoch 9/250
13/13 [==============================] - 0s 2ms/step - loss: 1.1814 - accuracy: 0.6349
Epoch 10/250
13/13 [==============================] - 0s 2ms/step - loss: 1.1113 - accuracy: 0.6508
Epoch 11/

In [64]:
model.save("chatbot_model.h5", history)